😵 simple __import__s for notebooks and markdown files.  All code elements are executed in order and imported into sys models.

In [1]:
        __all__ = 'finder',
        meta_paths = []

In [2]:
        from importlib.util import spec_from_loader
        from importlib.machinery import SourceFileLoader, FileFinder
        from nbconvert.exporters.base import export, get_exporter
        import sys
        from os.path import sep, curdir, extsep, exists
        from nbconvert import get_exporter

In [3]:
        class Importable(object):
            def find_spec(self, name, paths, target=None):
                for ext in type(self.ext) is str and [self.ext] or self.ext:
                    for path in paths or [curdir]:
                        path = extsep.join([sep.join([path, name.split('.')[-1]]), ext])
                        if exists(path):
                            return spec_from_loader(name, type(self)(name, path))
                return None

In [4]:
        exporter = get_exporter('python')()

In [5]:
        def add_finder(finder):
            sys.meta_path.append(finder("", ""))
            meta_paths.append(sys.meta_path[-1])
            return finder

In [6]:
        def finder(ext, bases=[SourceFileLoader], **kwargs):
            """A decorator to create a new ext loader by defining the get_code method.
            """
            def importer(func):
                return add_finder(type(
                    func.__name__, tuple([Importable] + bases), 
                    kwargs.update(get_code=func, ext=ext) or kwargs
                ))
            return importer

In [7]:
        def Ipynb(self, path):
            return exporter.from_filename(self.path)[0].encode('utf-8')

As proof-of-concept, create a finder for ipynb files.

In [8]:
        def load_ipython_extension(ip=None):
            finder('ipynb')(Ipynb)
            sys.path_importer_cache.clear()

In [9]:
        load_ipython_extension()

In [10]:
        import importable
        importable.__file__

'./importable.ipynb'

Then use that finder to load subsequent notebooks.

In [11]:
        def unload_ipython_extension(ip=None):
            for _ in sys.meta_path:
                if _ in meta_paths: sys.meta_path.pop(sys.meta_path.index(_))
            for _ in sys.path_hooks:
                if _ in path_hooks: sys.path_hooks.pop(sys.path_hooks.index(_))
            sys.path_importer_cache.clear()

In [12]:
        if False and __name__ == '__main__': 
            !jupyter nbconvert --to script importable.ipynb